In [2]:
# Main
import os
import shutil
import cv2
import numpy as np 
import pandas as pd
import glob
from tqdm import tqdm
tqdm.pandas()
import re

# Data Visualization
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import Image, display

import torch
from numba import cuda

In [40]:
provinces = ["皖", "沪", "津", "渝", "冀", "晋", "蒙", "辽", "吉", "黑", "苏", "浙", "京", "闽", "赣", "鲁", "豫", "鄂", "湘", "粤", "桂", "琼", "川", "贵", "云", "藏", "陕", "甘", "青", "宁", "新", "警", "学", "O"]
alphabets = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W',
             'X', 'Y', 'Z', 'O']
ads = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X',
       'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'O']

# list_sub = ["ccpd_base", "ccpd_fn", "ccpd_db", "ccpd_rotate", "ccpd_weather", "ccpd_blur"]
list_sub = ["ccpd_base"]
BASE_PATH = "C:/Users/User/DeepLearning/Deep_Learning/final_project"

In [1]:
file = "C:/Users/User/DeepLearning/Deep_Learning/final_project/ccpd_base/01-86_91-298,341_449,414-458,394_308,410_304,357_454,341-0_0_14_28_24_26_29-124-24.jpg"

In [41]:
img_path = []
left_up_x = []
left_up_y = []
right_down_x = []
right_down_y = []
decoded_plate = []
for sub in list_sub:
    sub_path = os.path.join(BASE_PATH, sub)
    sub_files = glob.glob(os.path.join(sub_path, "*"))

    for file in sub_files:

        img_path.append(file)

        file_no_jpg = file.split("/")[-1][:-4]

        boundind_box = file_no_jpg.split("-")[2]
        left_up_bbox = boundind_box.split("_")[0]
        right_down_bbox = boundind_box.split("_")[1]
        left_up_x .append(left_up_bbox.split(",")[0])
        left_up_y .append(left_up_bbox.split(",")[1])
        right_down_x .append(right_down_bbox.split(",")[0])
        right_down_y .append(right_down_bbox.split(",")[1])



        temp = []
        plate_number =  file_no_jpg.split("-")[-3]
        plate_number_split =  plate_number.split('_')
        city = provinces[int(plate_number_split[0])]
        alp = alphabets[int(plate_number_split[1])]
        temp.append(city)
        temp.append(alp)
        
        for i in range(len(plate_number_split)-2):
            temp.append(ads[int(plate_number_split[i+2])])
        k = "".join(temp)
        decoded_plate.append(k)

r = [img_path,left_up_x,left_up_y,right_down_x,right_down_y,decoded_plate]
all_data = pd.DataFrame(r)
all_data = all_data.T
all_data.columns = ['img_path','x1','y1','x2','y2','decode_plate']
all_data                

,img_path,x1,y1,x2,y2,decode_plate
0,C:/Users/User/DeepLearning/Deep_Learning/final...,352,516,448,547,皖AYL250
1,C:/Users/User/DeepLearning/Deep_Learning/final...,283,519,381,553,皖AH2T95
2,C:/Users/User/DeepLearning/Deep_Learning/final...,441,517,538,546,皖APS969
3,C:/Users/User/DeepLearning/Deep_Learning/final...,351,564,451,606,皖ADX640
4,C:/Users/User/DeepLearning/Deep_Learning/final...,341,443,436,482,皖AK9954
...,...,...,...,...,...,...
199991,C:/Users/User/DeepLearning/Deep_Learning/final...,27,426,677,670,皖G95192
199992,C:/Users/User/DeepLearning/Deep_Learning/final...,78,438,602,676,皖A8R888
199993,C:/Users/User/DeepLearning/Deep_Learning/final...,0,398,691,606,皖A92R98
199994,C:/Users/User/DeepLearning/Deep_Learning/final...,104,414,655,665,皖AQK091
